In [196]:
from math import sqrt
from tabulate import tabulate
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.linalg import sqrtm

# Vandermonde matrix is completely positive. See prove here: https://scask.ru/a_book_matrix.php?id=104
def generate_generalized_vandermonde_matrix(n: int) -> np.array:
    A = np.zeros((n, n), float)
    for i in range(n):
        a_i = i + 1
        for j in range(n):
            b_j = (j + 1) / 2
            A[i, j] = a_i ** b_j
    return A


def get_first_regularization_method_A_u(A: np.array, u: np.array, alfa: float):
    A = np.matrix(A)
    n = A.shape[0]
    new_A = A.H @ A + np.eye(n) * alfa
    new_u = A.H @ u
    return np.array(new_A), np.array(new_u).flatten()

def get_second_regularization_method_B_u(B: np.array, u: np.array, alfa: float):
    B = np.matrix(B)
    n = B.shape[0]
    inverse_B = np.linalg.inv(B)
    new_B = B.H @ B + np.eye(n) * alfa
    first = np.array(inverse_B @ u).flatten()
    new_u = B.H @ first
    return np.array(new_B), np.array(new_u).flatten()


def find_sqrt_root_of_matrix(A):
    eigen_values, eigen_matrix = np.linalg.eig(A)
    assert sorted(eigen_values) == sorted(list(set(eigen_values))), f"Eigen values repeats! \n {eigen_values}"
    assert len([value for value in eigen_values if value <= 0]) == 0, f"Eigen value is negative! {n} \n {eigen_values}"

    lambda_matrix = np.zeros((n, n))
    for i in range(n):
        lambda_matrix[i, i] = eigen_values[i]
    sqrt_lambda_matrix = np.zeros((n, n))
    for i in range(n):
        sqrt_lambda_matrix[i, i] = sqrt(eigen_values[i])

    B = eigen_matrix.dot(sqrt_lambda_matrix.dot(np.linalg.inv(eigen_matrix)))
    return B

def pretty_print(A, precision):
    np.set_printoptions(precision=precision)
    print(A)

In [248]:
results = []
header = ["n", "cond(A)", "cond(B)", "norm(A, B^2)"]
for n in range(2, 15):
    A = generate_generalized_vandermonde_matrix(n)
    B = find_sqrt_root_of_matrix(A)
    result = [n, np.linalg.cond(A), np.linalg.cond(B), np.linalg.norm(A - np.linalg.matrix_power(B, 2))]
    results.append(result)
print(tabulate(results, header))

  n           cond(A)           cond(B)    norm(A, B^2)
---  ----------------  ----------------  --------------
  2      13.5832            3.69881         5.43896e-16
  3     177.293            13.3365          8.782e-15
  4    2716.28             52.3533          2.56356e-14
  5   48854.1             224.688           3.80463e-14
  6  994899              1035.8             2.71223e-13
  7       2.23069e+07    5052.76            3.59352e-13
  8       5.40682e+08   25817.5             1.93369e-12
  9       1.3993e+10   137208               2.10821e-11
 10       3.83251e+11  754577               7.23629e-11
 11       1.10341e+13       4.27775e+06     1.06026e-09
 12       3.32409e+14       2.49236e+07     3.99535e-09
 13       1.05704e+16       1.49727e+08     2.47531e-08
 14       3.57625e+17       9.24135e+08     1.77047e-07


In [241]:
regularization_data = []
columns = ["n", "regularized_matrix", "cond_value", "discrepancy", "regularization_method"]
alfa = 0.00000000001
for i in range(1, 50):
    bad_n = i
    A = generate_generalized_vandermonde_matrix(bad_n)
    B = sqrtm(A)
    standard_vector = np.ones(bad_n)
    u = A @ standard_vector

    new_A, new_u = get_first_regularization_method_A_u(A, u, alfa)
    discrepancy_A = np.linalg.norm(np.linalg.solve(new_A, new_u) - standard_vector)
    regularization_data.append([bad_n, new_A, np.linalg.cond(new_A), discrepancy_A, "first"])

    new_B, new_u = get_second_regularization_method_B_u(B, u, alfa)
    discrepancy_B = np.linalg.norm(np.linalg.solve(new_B, new_u) - standard_vector)
    regularization_data.append([bad_n, new_B, np.linalg.cond(new_B), discrepancy_B, "second"])
df = pd.DataFrame(regularization_data, columns=columns)

fig = px.line(df, x="n", y="discrepancy", color="regularization_method")
fig.show()

In [245]:
columns = ["alfa", "regularized_matrix", "cond_value", "discrepancy", "regularization_method"]
bad_n = 23
regularization_data = []
for i in range(-5, 6):
    alfa = 10 ** i
    A = generate_generalized_vandermonde_matrix(bad_n)
    B = sqrtm(A)
    standard_vector = np.ones(bad_n)
    u = A @ standard_vector

    new_A, new_u = get_first_regularization_method_A_u(A, u, alfa)
    discrepancy_A = np.linalg.norm(np.linalg.solve(new_A, new_u) - standard_vector)
    regularization_data.append([alfa, new_A, np.linalg.cond(new_A), discrepancy_A, "first"])

    new_B, new_u = get_second_regularization_method_B_u(B, u, alfa)
    discrepancy_B = np.linalg.norm(np.linalg.solve(new_B, new_u) - standard_vector)
    regularization_data.append([alfa, new_B, np.linalg.cond(new_B), discrepancy_B, "second"])
df = pd.DataFrame(regularization_data, columns=columns)

fig = px.line(df, x="alfa", y="discrepancy", color="regularization_method")
fig.show()

In [132]:
df

,n,regularized_matrix,cond_value,discrepancy,regularization_method
0,23,"[[276.001, 1070.5240113105895, 4324.0, 17963.1...",3.091151e+34,4.069893e+25,first
1,23,"[[(1.154259662846738+6.837829584231086e-15j), ...",1.038597e+19,4.246184e+05,second
2,24,"[[300.001, 1188.099518964182, 4900.0, 20784.97...",1.062470e+36,5.487782e+26,first
3,24,"[[(0.25203805751036656-0.6147127497140871j), (...",5.097570e+19,1.692757e+06,second
4,25,"[[325.001, 1313.099518964182, 5525.0, 23909.97...",6.170566e+36,7.007527e+29,first
...,...,...,...,...,...
149,97,"[[(1.001000000000001-1.2726005562886512e-42j),...",7.938910e+76,2.196622e+48,second
150,98,"[[4851.001, 38516.18695258338, 318549.0, 27098...",2.443046e+140,inf,first
151,98,"[[(1.001-4.055790367664331e-42j), (8.825794687...",4.148397e+80,1.068639e+49,second
152,99,"[[4950.001, 39501.224515318936, 328350.0, 2807...",3.881410e+141,inf,first
